In [ ]:
import numpy as np
import pandas as pd 
import logging

from gensim.models import Word2Vec


from words_parser import ingredient_parser

In [ ]:
 
data = pd.read_csv(r'C:\xampp\htdocs\3161Database files\recipe_recommender\app\csvfiles\parsed_recipesv4.csv')
data['parsed_ingredients'] = data.RecipeIngredientParts.apply(ingredient_parser)
data.head()

In [ ]:
# get corpus with the documents sorted in alphabetical order
def get_and_sort_ingr(data):
    ingreds_sorted = []
    for doc in data.parsed_ingredients.values:
        ingreds_sorted.append(doc)
    return ingreds_sorted

ingreds = get_and_sort_ingr(data[:5500])
#for i in ingreds:
#    print(i)
print(f"Length of ingreds: {len(ingreds)}")

In [ ]:
thirdofcorpus = round(len(ingreds)*.25)
#print(len(ingreds))
print(len(ingreds[:thirdofcorpus]))

lengths = [len(doc) for doc in ingreds]
print(round(sum(lengths)/len(lengths)))
avg_len = float(sum(lengths)) / len(lengths)
avg_len


In [ ]:
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 8 # number of CPUs
window = 6 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow = Word2Vec(ingreds, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100)

In [ ]:
#Summarize the loaded model
print(model_cbow)

#Summarize vocabulary
words = list(model_cbow.wv.index_to_key)
words.sort()
print(words)

#Acess vector for one word
print("check ",model_cbow.wv['tomato'])

# most similar
model_cbow.wv.most_similar(u'tomato')
#print("after stock")
#model_cbow.wv.similarity('cauliflower', 'cauliflower just larger than potato')

In [ ]:
model_cbow.save('models/model_cbow_ingredients.bin')

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

X = model_cbow[model_cbow.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)

fig = plt.figure(figsize=(100,60))

plt.scatter(result[:300, 0], result[:300, 1])
words = list(model_cbow.wv.vocab)[:300]
for i, word in enumerate(words):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]), fontsize=15)
plt.show()